3º Problem - Image classification

In [9]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import keras
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.callbacks import EarlyStopping
from keras_tuner import HyperModel, RandomSearch
from keras import layers, Sequential
from keras.callbacks import EarlyStopping
from keras.models import load_model
from imblearn.under_sampling import RandomUnderSampler
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import confusion_matrix
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import classification_report, confusion_matrix



    Class Weights

In [10]:
def weigthed_classes(y_train):

    class_labels = np.unique(y_train)
    class_weights = compute_class_weight(class_weight='balanced', classes=class_labels, y=y_train)
    class_weight_dict = {i: class_weights[i] for i in range(len(class_labels))}

    return class_weight_dict


    Oversampling (SMOTE)

- Generates synthetic examples rather than just duplicating existing minority class samples.
- Helps improve the model's ability to generalize by providing diverse training data.

In [11]:
from imblearn.over_sampling import SMOTE
from keras.utils import to_categorical

def oversample_with_smote(X_train, y_train,Y_train):
    
    num_classes = len(np.unique(Y_train))
    smote = SMOTE(random_state=42)
    X_train_flat = X_train.reshape(X_train.shape[0], -1) 
    X_train_os, y_train_os = smote.fit_resample(X_train_flat, y_train)
    X_train_os = X_train_os.reshape(X_train_os.shape[0], 48, 48, 1)  
    y_train_os = to_categorical(y_train_os, num_classes=num_classes) 
    return X_train_os, y_train_os


    Undersampling

In [12]:
def undersampling(X_train,y_train):
    
    X_train_flat = X_train.reshape(X_train.shape[0], -1)

    rus = RandomUnderSampler(random_state=42)
    X_train_us, y_train_us = rus.fit_resample(X_train_flat, y_train)

    X_train_us = X_train_us.reshape(X_train_us.shape[0], 48, 48, 1)

    y_train_us= to_categorical(y_train_us, 2)

    return X_train_us, y_train_us


    Model Evaluation

In [13]:
def evaluate_model(cm):   
    tn = cm[0, 0]  # True negatives
    fp = cm[0, 1]  # False positives
    fn = cm[1, 0]  # False negatives
    tp = cm[1, 1]  # true positives

    precision = tp / (tp + fp) if (tp + fp) > 0 else 0
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0
    f1_score1 = (2 * precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    print(f1_score1)

    precision = tn/(tn+fn) if (tn+fn) > 0 else 0
    recall = tn/(tn+fp) if tn+fp >0 else 0
    f1_score2 = (2 * precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    print(f1_score2)

    f1_score  = (f1_score1 + f1_score2) / 2
    accuracy = (tp + tn) / (tp + fp + fn + tn)
    specificity = tn / (tn + fp) if (tn + fp) > 0 else 0
    sensitivity = recall  
    balanced_accuracy = (specificity + sensitivity) / 2

    print(f"f1 score:{f1_score}")
    print(f"accuracy:{accuracy}")
    print(f"balanced_accuracy:{balanced_accuracy}")


    Data balancing

In [ ]:
OS = True
US = False
WC = False

X_train = np.load("Xtrain1.npy")  
Y_train = np.load("Ytrain1.npy")  
X_test = np.load("Xtest1.npy")

X_train_normalized = X_train.astype('float32') / 255.0

X_train, X_val, y_train, y_val = train_test_split(X_train_normalized, Y_train, test_size=0.2, random_state=42)

num_classes = len(np.unique(Y_train))

y_val = keras.utils.to_categorical(y_val, num_classes=num_classes)

if US:
    print("undersampling")
    X_train_us, y_train_us = undersampling(X_train, y_train)
    y_train_us = keras.utils.to_categorical(np.argmax(y_train_us, axis=1), num_classes=num_classes)

elif OS:
    print("oversampling")
    X_train_os, y_train_os = oversample_with_smote(X_train, y_train,Y_train)
    y_train_os = keras.utils.to_categorical(np.argmax(y_train_os, axis=1), num_classes=num_classes)

elif WC:
    print("weighted classes")
    class_weight_dict = weigthed_classes(y_train)
    print(class_weight_dict)
    y_train = keras.utils.to_categorical(y_train, num_classes=num_classes)
else:
    print("imbalanced")
    y_train = keras.utils.to_categorical(y_train, num_classes=num_classes)


    1.CNN 

In [ ]:
class CNNHyperModel(HyperModel):
    def build(self, hp):
        model = Sequential()

        model.add(layers.Conv2D(
            filters=hp.Int('filters1', min_value=32, max_value=128, step=32),
            kernel_size=(3, 3),
            activation='relu',
            input_shape=(48, 48, 1)
        ))
        model.add(layers.MaxPooling2D((2, 2)))
        model.add(layers.Dropout(hp.Float('dropout1', min_value=0.2, max_value=0.5, step=0.1)))  # Dropout layer

        # Second convolutional layer
        model.add(layers.Conv2D(
            filters=hp.Int('filters2', min_value=32, max_value=128, step=32),
            kernel_size=(3, 3),
            activation='relu'
        ))
        model.add(layers.MaxPooling2D((2, 2)))
        model.add(layers.Dropout(hp.Float('dropout2', min_value=0.2, max_value=0.5, step=0.1)))  

        # Third convolutional layer
        model.add(layers.Conv2D(
            filters=hp.Int('filters3', min_value=32, max_value=128, step=32),
            kernel_size=(3, 3),
            activation='relu'
        ))
        model.add(layers.MaxPooling2D((2, 2)))
        model.add(layers.Dropout(hp.Float('dropout3', min_value=0.2, max_value=0.5, step=0.1)))  

        # Flatten and add Dense layers
        model.add(layers.Flatten())
        model.add(layers.Dense(units=hp.Int('units', min_value=64, max_value=256, step=64), activation='relu'))
        model.add(layers.Dropout(0.5)) 
        model.add(layers.Dense(num_classes, activation='softmax'))  

        # Compile the model
        model.compile(optimizer=keras.optimizers.Adam(learning_rate=hp.Float('learning_rate', min_value=1e-4, max_value=1e-2, sampling='LOG')),
                    loss='categorical_crossentropy',
                    metrics=['accuracy'])
        return model

# Initialize the HyperModel
hypermodel = CNNHyperModel()

# Define the tuner
tuner = RandomSearch(
    hypermodel,
    objective='val_accuracy',
    max_trials=10,
    executions_per_trial=1,
    directory='hyperparam_tuning',
    project_name='cnn_tuning'
)

early_stopping = EarlyStopping(monitor='val_loss',
                            patience=5,
                            restore_best_weights=True)

X_train_reshaped = X_train.reshape(-1, 48, 48, 1)
X_val_reshaped = X_val.reshape(-1, 48, 48, 1)

if OS:
    print("OVERSAMPLING")
    X_train_os_reshaped = X_train_os.reshape(-1, 48, 48, 1)
    tuner.search(X_train_os_reshaped, y_train_os, epochs=20, validation_data=(X_val_reshaped, y_val), callbacks=[early_stopping])
    best_model = tuner.get_best_models(num_models=1)[0]
    best_model.fit(X_train_os_reshaped, y_train_os, epochs=100, validation_data=(X_val_reshaped, y_val), callbacks=[early_stopping])
    best_model.save('cnn_model_os.keras')
    val_loss, val_accuracy = best_model.evaluate(X_val_reshaped, y_val)
    print(f"Validation Loss: {val_loss}, Validation Accuracy: {val_accuracy}")

elif US:
    print("UNDERSAMPLING")
    X_train_us_reshaped = X_train_us.reshape(-1, 48, 48, 1)
    tuner.search(X_train_us_reshaped, y_train_us, epochs=20, validation_data=(X_val_reshaped, y_val), callbacks=[early_stopping])
    best_model = tuner.get_best_models(num_models=1)[0]
    best_model.fit(X_train_us_reshaped, y_train_us, epochs=20, validation_data=(X_val_reshaped, y_val), callbacks=[early_stopping])
    best_model.save('cnn_model_us.keras')
    val_loss, val_accuracy = best_model.evaluate(X_val_reshaped, y_val)
    print(f"Validation Loss: {val_loss}, Validation Accuracy: {val_accuracy}")

elif WC:
    print("WEIGHTED CLASSES")
    X_train_normalized_reshaped = X_train_normalized.reshape(-1, 48, 48, 1)
    tuner.search(X_train_reshaped, y_train, epochs=20, validation_data=(X_val_reshaped, y_val), callbacks=[early_stopping])
    best_model = tuner.get_best_models(num_models=1)[0]
    best_model.fit(X_train_reshaped, y_train, epochs=20,class_weight=class_weight_dict ,validation_data=(X_val_reshaped, y_val), callbacks=[early_stopping])
    best_model.save('cnn_model_wc.keras')
    val_loss, val_accuracy = best_model.evaluate(X_val_reshaped, y_val)
    print(f"Validation Loss: {val_loss}, Validation Accuracy: {val_accuracy}")

else:
    print("IMBALANCED")
    tuner.search(X_train_reshaped, y_train, epochs=20, validation_data=(X_val_reshaped, y_val), callbacks=[early_stopping])
    best_model = tuner.get_best_models(num_models=1)[0]
    best_model.fit(X_train_reshaped, y_train, epochs=20, validation_data=(X_val_reshaped, y_val), callbacks=[early_stopping])
    best_model.save('cnn_model_imbalanced.keras')
    val_loss, val_accuracy = best_model.evaluate(X_val_reshaped, y_val)
    print(f"Validation Loss: {val_loss}, Validation Accuracy: {val_accuracy}")


    Confusion Matrix

In [ ]:
#---------------------------------------------------------------------------------------------
#                                CNN - Model prediction
#---------------------------------------------------------------------------------------------

print("CNN")

if OS:
    print("oversampling")
    model = load_model('best_cnn_model.keras')

elif US:
    print("undersampling")
    model = load_model('cnn_model_us.keras')

elif WC:
    print("weighted classes")
    model = load_model('cnn_model_wc.keras')

else:
    print("imbalanced")
    model = load_model('cnn_model_imbalanced.keras')

X_test_normalized = X_test.astype('float32') / 255.0
X_test_normalized = X_test_normalized.reshape(-1, 48, 48, 1)

y_pred = model.predict(X_test_normalized)
y_pred = np.argmax(y_pred, axis=1)
np.save('Ytest1.npy',y_pred)

X_val = X_val.reshape(-1, 48, 48, 1)
binary_predictions = model.predict(X_val)

binary_predictions = np.argmax(binary_predictions, axis=1)

y_val_reshaped = np.argmax(y_val, axis=1)

#---------------------------------------------------------------------------------------------
#                                   CONFUSION MATRIX
#---------------------------------------------------------------------------------------------
print(classification_report(binary_predictions, y_val_reshaped, digits = 6))
cm = confusion_matrix(binary_predictions, y_val_reshaped)

if cm is not None:

    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
                xticklabels=np.unique(Y_train), 
                yticklabels=np.unique(Y_train))
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.title('Confusion Matrix')
    plt.show()

    print('Confusion Matrix:')
    print(cm)   
    evaluate_model(cm)
else:
    print("A matriz de confusão não foi criada. Verifique as condições.")
    

    Image Visualization

In [ ]:
def display_image_from_npy(file_path, index, image_shape):

    data = np.load("Xtest1.npy")

    image_vector = data[20]

    image = image_vector.reshape(image_shape)
    
    image = np.squeeze(image)
    
    plt.imshow(image, cmap='gray') 
    plt.title(f"Image at Index {index}")
    plt.axis('off')
    plt.show()

file_path = 'Xtrain1.npy' 
index = 16 
image_shape = (48, 48, 1) 

display_image_from_npy(file_path, index, image_shape)
